Test action space sample

In [2]:
from sb3_contrib import MaskablePPO
import edge_relocation as er
import json
import numpy as np


Test masks

In [3]:
max_trajectory_length = 25
min_trajectory_length = 25
initial_load = "low" # low (10-40%), medium(40-60%)), high(60-80%), random (10-80%), variable_load ( different initial load for each episode)

# create environment
erEnv = er.EdgeRelEnv("topoconfig.json", min_trajectory_length, max_trajectory_length, initial_load)
erEnv.reset()
state = erEnv.get_state()
print("STATE:\n")
print(state)
print("JSON:\n")

for key in state:
    state[key] = state[key].tolist()

json_data = json.dumps(state)

print(json_data)

# Deserialize JSON string to Python dictionary
my_deserialized_dict = json.loads(json_data)

# Convert Python lists to NumPy arrays
for key in my_deserialized_dict:
    my_deserialized_dict[key] = np.array(my_deserialized_dict[key])

print("deserialized STATE:\n")
print(my_deserialized_dict)



STATE:

{'space_App': array([[ 2.,  5.,  3.,  9., 15.]]), 'space_MEC': array([[ 1., 62.,  1., 77.,  3.],
       [ 1., 95.,  1., 89.,  3.],
       [ 1., 72.,  1., 68.,  3.],
       [ 1., 75.,  1., 81.,  3.],
       [ 1., 74.,  1., 67.,  3.],
       [ 1., 77.,  1., 85.,  3.],
       [ 1., 70.,  1., 90.,  3.],
       [ 1., 79.,  1., 73.,  3.],
       [ 1., 63.,  1., 89.,  3.],
       [ 1., 66.,  1., 84.,  3.],
       [ 1., 82.,  1., 73.,  3.],
       [ 1., 85.,  1., 64.,  3.],
       [ 1., 81.,  1., 73.,  3.],
       [ 1., 86.,  1., 67.,  3.],
       [ 1., 88.,  1., 84.,  3.],
       [ 1., 65.,  1., 63.,  3.],
       [ 2., 87.,  2., 85.,  2.],
       [ 2., 72.,  2., 85.,  2.],
       [ 2., 66.,  2., 87.,  2.],
       [ 2., 72.,  2., 81.,  2.],
       [ 2., 63.,  2., 65.,  2.],
       [ 3., 72.,  3., 65.,  1.]])}
JSON:

{"space_App": [[2.0, 5.0, 3.0, 9.0, 15.0]], "space_MEC": [[1.0, 62.0, 1.0, 77.0, 3.0], [1.0, 95.0, 1.0, 89.0, 3.0], [1.0, 72.0, 1.0, 68.0, 3.0], [1.0, 75.0, 1.0, 81.0, 3.0]

In [4]:
erEnv.reset()
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for mec in erEnv.mec_nodes:
    print(mec.id, "cpu util: ", mec.cpu_utilization, "mem util: ", mec.memory_utilization)
print(erEnv.action_masks())

#please note that masks refers to the valid action of selecting 'mec1', mec2, mec3 etc, while the list below lists mecs in wrong order

App reqlatency:  25 current MEC ID:  mec1 current cell ID:  12
mec8 cpu util:  87 mem util:  81
mec9 cpu util:  71 mem util:  81
mec10 cpu util:  73 mem util:  84
mec7 cpu util:  72 mem util:  82
mec12 cpu util:  60 mem util:  82
mec13 cpu util:  83 mem util:  64
mec14 cpu util:  85 mem util:  74
mec11 cpu util:  83 mem util:  87
mec16 cpu util:  88 mem util:  63
mec17 cpu util:  83 mem util:  76
mec18 cpu util:  67 mem util:  76
mec15 cpu util:  81 mem util:  63
mec20 cpu util:  72 mem util:  82
mec21 cpu util:  79 mem util:  66
mec22 cpu util:  66 mem util:  81
mec19 cpu util:  67 mem util:  79
mec2 cpu util:  86 mem util:  87
mec3 cpu util:  76 mem util:  68
mec4 cpu util:  69 mem util:  66
mec5 cpu util:  87 mem util:  78
mec6 cpu util:  75 mem util:  81
mec1 cpu util:  70 mem util:  82


AttributeError: 'EdgeRelEnv' object has no attribute 'action_masks'

Test Maskable PPO

In [11]:
import edge_relocation as er

import datetime


# create environment
erEnv = er.EdgeRelEnv("topoconfig.json")
# obs = erEnv.reset()
log_dir = "../logs/PPO/" + datetime.datetime.now().strftime("Test_Mask_high_load_%Y%m%d-%H%M%S")
model = MaskablePPO("MultiInputPolicy", erEnv, verbose=0, tensorboard_log = log_dir, n_steps=64)
model.learn(total_timesteps=5000)
print("episodes number: ", erEnv.episodes_counter)


#Seems that for today, MaskablePPO is checking about the type of Space, and it accepts only spaces of gym, not yet a spaces of gymnasium. Further checks required

ValueError: Environment does not support action masking. Consider using ActionMasker wrapper

5566545+454+5412384049++8049+Test manually single episode

In [17]:
erEnv = er.EdgeRelEnv("topoconfig.json")
obs = erEnv.reset()
print("trajectory (len) : ", len(erEnv.trajectory) ,erEnv.trajectory)
print("App reqlatency: ", erEnv.mecApp.app_req_latency, "current MEC ID: ", erEnv.mecApp.current_MEC.id, "current cell ID: ", erEnv.mecApp.user_position)
for i in range (len(erEnv.trajectory)):
    print("masks:", erEnv.action_masks())
    print("current_cell:", erEnv.trajectory[i])
    obs, rewards, dones, info = erEnv.step(3)
    print(obs)
    print(rewards)
    print(dones)

trajectory (len) :  11 [2, 14, 16, 23, 35, 37, 42, 38, 28, 21, 17]
App reqlatency:  10 current MEC ID:  mec12 current cell ID:  2
masks: [False, False, False, False, False, False, True, True, False, False, True, True, False, False, False, False, False, False, False, False, False, False]
current_cell: 2
{'space_App': array([[ 1.,  2.,  1.,  4., 14.]]), 'space_MEC': array([[ 1., 29.,  1., 29.,  3.],
       [ 1., 88.,  1., 88.,  3.],
       [ 1., 81.,  1., 81.,  3.],
       [ 1., 27.,  1., 27.,  3.],
       [ 1.,  0.,  1.,  1.,  3.],
       [ 1., 89.,  1., 89.,  3.],
       [ 1., 83.,  1., 83.,  3.],
       [ 1., 19.,  1., 19.,  3.],
       [ 1., 14.,  1., 14.,  3.],
       [ 1., 87.,  1., 87.,  3.],
       [ 1., 10.,  1., 10.,  3.],
       [ 1., 63.,  1., 63.,  3.],
       [ 1., 17.,  1., 17.,  3.],
       [ 1., 63.,  1., 63.,  3.],
       [ 1., 67.,  1., 67.,  3.],
       [ 1., 72.,  1., 72.,  3.],
       [ 2., 49.,  2., 49.,  2.],
       [ 2., 10.,  2., 10.,  2.],
       [ 2.,  1.,  2.